## 0. Environment Setup
We create a smiliar environemnt that simulate the inputs and variables as per in ciop jobs

In [64]:
#old input1='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=230286758E25FD953AF736DE85F01CDF9C539EC7' #niger-hype-model.zip, one file when unzipped
#old input2='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9345ED73B72F49E6FF31B07B57013BC519210E24' #niger-hype-model-2.23.zip, one dir when unzipped
input2='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9345ED73B72F49E6FF31B07B57013BC519210E24' #niger-hype-model-2.23.zip, one dir when unzipped
#use this from model.zip instead  input_hype_data='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9CE420998D9EC367470EEAEEA6B6DA0B0E4604D2' #niger-hype-data.zip, dirs when unzipped
TMPDIR="/workspace/tmpdir"
TMPDIR

[1] "/workspace/tmpdir"

1. Read input model
This inital step collect via proper opensearch call and copy the HYPE model config archive

In [65]:
app.date = format(Sys.time(), "%Y%m%d_%H%M")

## set application name
app.name = "forecast"
## ------------------------------------------------------------------------------
## flag which environment is used, if not set
if(!exists("app.sys")){
    app.sys ="tep"
    }
## ------------------------------------------------------------------------------
## load rciop package and set working directory to TMPDIR when running on TEP 
if(app.sys=="tep"){
    library("rciop")

    rciop.log ("DEBUG", " *** hypeapps-forecast *** TEP hydrological modelling applications ***", "/node_forecast/run.R")
    rciop.log ("DEBUG", " rciop library loaded", "/node_forecast/run.R")

    setwd(TMPDIR)
    rciop.log("DEBUG", paste(" R session working directory set to ",TMPDIR,sep=""), "/node_forecast/run.R")
}

Some rciop functions may need to be overridden in the notebook environment.
Set to your current needs.

In [66]:
ENABLE_STUB_RCIOP_LOG <- FALSE
#ENABLE_STUB_RCIOP_LOG <- TRUE

if (ENABLE_STUB_RCIOP_LOG == TRUE) {
    message("rciop.log() overridden by local stub function...")
    
    rciop.log <- function(type, message, proc='') {

        type <- type
        msg <- message
    if (proc=='') { proc <- "user process" }

    out.msg <- paste(format(Sys.time(), format="%Y-%m-%dT%H:%M:%S"), "[", type, "][", proc, "] ", msg, sep="")

    message(out.msg)
    } # rciop.log

} # end stub

In [67]:
#ENABLE_STUB_RCIOP_GETPARAM <- FALSE
ENABLE_STUB_RCIOP_GETPARAM <- TRUE

if (ENABLE_STUB_RCIOP_GETPARAM == TRUE) {
    message("rciop.getparam() overridden by local stub function...")
    # Change according to your test objective.
    # Would normally return values defined in application.xml for specified ids'.
    
    rciop.getparam <- function(id) {
        value <- NULL # or ""
        
        if (id == "model") {
            value <- "https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=230286758E25FD953AF736DE85F01CDF9C539EC7"
        }
        else if (id == "idate" ) {
            value <- "2018-09-01"
        }
        else {
            message("rciop.get_param:id not found...")
        }
        
        return (value)
    } # rciop.getparam

} # end stub

rciop.getparam() overridden by local stub function...


Add your code to test...
Remove current, or paste other code or add additional code blocks.

WL

In [135]:
# opensearch-client -p 'count=unlimited' -p 'cat=dataitem' "https://catalog.terradue.com/fanfar-00001/search" enclosure
#wl_query <- 'opensearch-client -p 'count=unlimited' -p 'cat=dataitem' "https://catalog.terradue.com/fanfar-00001/search"'

In [185]:
#rciop.log("INFO", paste("Processing WL:", , sep=" "))

# Query the input reference
#message(paste("opensearch-client '",url_wl_query,"' enclosure", sep=""))
#opensearchCmd=paste("opensearch-client -p 'count=unlimited' -p 'cat=dataitem' "https://catalog.terradue.com/fanfar-00001/search" enclosure")
opensearchCmd=paste("opensearch-client -p 'count=5' -p 'cat=dataitem' 'https://catalog.terradue.com/fanfar-00001/search' enclosure")
input_enclosure <- system(command = opensearchCmd,intern = T)
if (length(input_enclosure > 1)) {
    #subdir <- 1
    n_url <- 0

    r_tmpdir <- paste(TMPDIR,"wl",sep="/")
    dir.create(r_tmpdir)

    for (url in 1:length(input_enclosure)) {
        #ok rciop.copy(paste0(input_enclosure[url], "/waterlevel.csv"), TMPDIR)
        #rciop.log("INFO", paste0(TMPDIR,url, sep="/"))
        #rciop.log("INFO", input_enclosure[url])
        #message(input_enclosure[url])
        #rciop.copy(paste0(input_enclosure[url], "/waterlevel.csv"), paste0(TMPDIR,str(subdir), sep="/"))
        
        #re <- "\/\[0\-9a\-zA\-Z\-\]\+\$"
        #re <- "\\[0-9a-zA-Z-\\]+W$"
        #re <- "([^/]+)/?$"
        #re <- ""
        #subdir <- regexpr(re, input_enclosure[url],perl=TRUE)
        #subdir <- regexpr("\\/\\[0-9a-zA-Z-\\]\\+\\$", input_enclosure[url])
        #subdir <- regexpr(([^/]+)/?$, input_enclosure[url])
        #subdir <- regexpr("[/^]+(?=/$|$)", input_enclosure[url])
        #subdir
        #message(paste(TMPDIR,subdir,sep="/"))
        
        #message(paste(TMPDIR,input_enclosure[url],sep="/"))
        #tmpdir <- paste(TMPDIR,input_enclosure[url],sep="/")
        #tmpdir <- TMPDIR
        n_url <- n_url + 1
        #tmpdir <- paste(TMPDIR,n_url,sep="/")
        #tmpdir <- paste(TMPDIR,"wl",sep="/")
        #dir.create(tmpdir)
        tmpdir <- paste(r_tmpdir,n_url,sep="/")
        #message(tmpdir)
        
        dir.create(tmpdir)
        url_wl <- paste(input_enclosure[url],"waterlevel.csv",sep="/")
        fh <- file(paste(tmpdir,"info.txt",sep="/"))
        write(input_enclosure[url], fh)
        close(fh)
        rciop.copy(url_wl,tmpdir)
        #rciop.copy(url_wl,TMPDIR)
        
        #rciop.copy(input_enclosure[url],TMPDIR)
        
        #rciop.copy(paste0(input_enclosure[url], "/waterlevel.csv"), paste0(TMPDIR,str(subdir), sep="/"))
        # kolla om filen verkligen finns
        #subdir <- subdir + 1
    }
}

[1] " -o /workspace/tmpdir/wl/1 https://store.terradue.com/fanfar-00001/_results/workflows/hydrology_tep_dcs_water_level_water_levels_4_0/run/9a839bb4-dad5-11e9-a7fe-0242ac110013/0006146-190709150342599-oozie-oozi-W/waterlevel.csv"
[1] " -o /workspace/tmpdir/wl/2 https://store.terradue.com/fanfar-00001/_results/workflows/hydrology_tep_dcs_water_level_water_levels_4_0/run/9bfdbf6a-dad5-11e9-9110-0242ac110013/0006147-190709150342599-oozie-oozi-W/waterlevel.csv"
[1] " -o /workspace/tmpdir/wl/3 https://store.terradue.com/fanfar-00001/_results/workflows/hydrology_tep_dcs_water_level_water_levels_4_0/run/e9d1db30-dacc-11e9-bbdc-0242ac110013/0006125-190709150342599-oozie-oozi-W/waterlevel.csv"
[1] " -o /workspace/tmpdir/wl/4 https://store.terradue.com/fanfar-00001/_results/workflows/hydrology_tep_dcs_water_level_water_levels_4_0/run/2a59bd5c-db4b-11e9-b85b-0242ac110013/0006326-190709150342599-oozie-oozi-W/waterlevel.csv"
[1] " -o /workspace/tmpdir/wl/5 https://store.terradue.com/fanfar-00001/

Same as above, but less text to manually change from json to when copying text from file later on.

In [187]:
# Query the input reference
#opensearchCmd=paste("opensearch-client -p 'count=unlimited' -p 'cat=dataitem' "https://catalog.terradue.com/fanfar-00001/search" enclosure")
opensearchCmd=paste("opensearch-client -p 'count=5' -p 'cat=dataitem' 'https://catalog.terradue.com/fanfar-00001/search' enclosure")
input_enclosure <- system(command = opensearchCmd,intern = T)
if (length(input_enclosure > 1)) {
    n_url <- 0

    r_tmpdir <- paste(TMPDIR,"wl",sep="/")
    dir.create(r_tmpdir)

    for (url in 1:length(input_enclosure)) {
        # Since not able to get end of url as dir name with regexpr
        n_url <- n_url + 1
        tmpdir <- paste(r_tmpdir,n_url,sep="/")
        dir.create(tmpdir)

        fh <- file(paste(tmpdir,"info.txt",sep="/"))
        write(input_enclosure[url], fh)
        close(fh)

        url_wl <- paste(input_enclosure[url],"waterlevel.csv",sep="/")

        # Download
        rciop.copy(url_wl,tmpdir)
    }
}


[1] " -o /workspace/tmpdir/wl/1 https://store.terradue.com/fanfar-00001/_results/workflows/hydrology_tep_dcs_water_level_water_levels_4_0/run/9a839bb4-dad5-11e9-a7fe-0242ac110013/0006146-190709150342599-oozie-oozi-W/waterlevel.csv"
[1] " -o /workspace/tmpdir/wl/2 https://store.terradue.com/fanfar-00001/_results/workflows/hydrology_tep_dcs_water_level_water_levels_4_0/run/9bfdbf6a-dad5-11e9-9110-0242ac110013/0006147-190709150342599-oozie-oozi-W/waterlevel.csv"
[1] " -o /workspace/tmpdir/wl/3 https://store.terradue.com/fanfar-00001/_results/workflows/hydrology_tep_dcs_water_level_water_levels_4_0/run/e9d1db30-dacc-11e9-bbdc-0242ac110013/0006125-190709150342599-oozie-oozi-W/waterlevel.csv"
[1] " -o /workspace/tmpdir/wl/4 https://store.terradue.com/fanfar-00001/_results/workflows/hydrology_tep_dcs_water_level_water_levels_4_0/run/2a59bd5c-db4b-11e9-b85b-0242ac110013/0006326-190709150342599-oozie-oozi-W/waterlevel.csv"
[1] " -o /workspace/tmpdir/wl/5 https://store.terradue.com/fanfar-00001/